## 7.13 循环

循环的效率取决于微处理器对循环控制分支预测的好坏。
有关分支预测的说明，请参阅前一段和手册3：“Intel，AMD和VIA CPU的微架构”。
对于循环次数固定且较小的循环，并且循环内部没有分支的情形，循环控制分支可以被完美的预测。
如上所述，可以预测的最大循环次数取决于处理器类型。
对于嵌套循环，
- 仅仅某些具有特殊的循环预测器的处理器可以预测得很好。
- 在其它处理器上，仅仅最内层循环可以被很好的预测。

重复计数次数很大的循环仅仅在退出循环是才被错误预测。
例如，如果一个循环重复一千次，那么循环控制分支在一千次中只被错误预测一次。因此错误预测惩罚对总执行时间的贡献可以忽略不计。

### 循环展开

在某些情况下，展开循环是一个优势。 例如：
```cpp
// Example 7.30a
int i;
for (i = 0; i < 20; i++) {
    if (i % 2 == 0) {
        FuncA(i);
    }
    else {
        FuncB(i);
    }
    FuncC(i);
}
```

该循环重复20次并交替调用`FuncA`和`FuncB`，然后调用`FuncC`。 将循环展开两次代码如下：
```cpp
// Example 7.30b
int i;
for (i = 0; i < 20; i += 2) {
    FuncA(i);
    FuncC(i);
    FuncB(i+1);
    FuncC(i+1);
}
```

此代码有三个优势：
- `i < 20`循环控制分支执行10次而不是20次。
- 重复次数从20减少到10，意味着它可以在奔腾4上完美预测。
- `if`分支被移除。

此循环展开也有下面的缺陷：
- 展开的循环在代码缓存或微操作缓存中占用更多空间。
- Core2处理器在非常小的循环（少于65个字节的代码）上表现更好。
- 如果重复次数是奇数并且你展开因子是2，那么必须在循环外进行额外的迭代。 通常，当重复次数不能被展开因子整除时，会出现此问题。

只有在可以获得特定优势的情况下才应使用循环展开。
如果循环包含浮点计算并且循环计数器是整数，那么通常可以假设整个计算时间由浮点代码而不是循环控制分支确定。
在这种情况下，通过展开循环没有任何好处。

在具有微操作高速缓存（例如，Sandy Bridge）的处理器上应该避免循环展开，因为重要的是节省微操作高速缓存的使用。
如果看上去有好处，编译器通常会自动展开循环（参见第72页）。

**程序员不必手动展开循环，除非能获得特别的好处，例如在示例`7.30b`中消除`if-branch`。**

### 循环控制条件

最有效的循环控制条件是一个简单的整数计数器。具有乱序功能的微处理器（参见第105页）将能够在几次迭代之前评估循环控制语句。
如果循环控制分支依赖于循环内的计算，则效率较低。
以下示例将以零结尾的ASCII字符串转换为小写：

```cpp
// Example 7.31a
char string[100], *p = string;
while (*p != 0) *(p++) |= 0x20;
```

如果已知字符串的长度，则使用循环计数器更有效：
```cpp
// Example 7.31b
char string[100], *p = string; int i, StringLength;
for (i = StringLength; i > 0; i--) *(p++) |= 0x20;
```

循环控制分支依赖于循环内部计算的常见情况是数学迭代，例如泰勒展开和牛顿迭代(牛顿-拉弗森迭代)。
这里重复迭代直到残余误差低于某个容差。
计算残余误差的绝对值并将其与容差进行比较，所花费的时间可能非常高，**以至于确定最坏情况最大重复计数，并始终使用该迭代次数效率更高。**

这种方法的优点是微处理器可以提前执行循环控制分支，并在循环内的浮点计算完成之前很久就解决任何分支错误预测。
如果典型的重复次数接近最大重复次数，并且每轮残差的计算时间相对总计算时间的贡献明显，则该方法是有利的。

循环计数器应该优先选择整数。如果循环需要浮点计数器，则创建一个额外的整数计数器。例如：
```cpp
// Example 7.32a
double x, n, factorial = 1.0;
for (x = 2.0; x <= n; x++) factorial *= x;
```
这可以**通过添加整数计数器，并在循环控制条件中使用整数来加以改进：**
```cpp
// Example 7.32b
double x, n, factorial = 1.0; int i;
for (i = (int)n - 2, x = 2.0; i >= 0; i--, x++) factorial *= x;
```

注意具有多个计数器的循环中逗号和分号之间的区别，如示例7.32b所示。
`for`循环有三个子句：初始化，条件和增量。这三个子句用分号分隔，而每个子句中的多个语句用逗号分隔。
**条件子句中应该只有一个语句。**
将**整数与零进行比较有时比将其与任何其他数字进行比较更有效**。
因此，将循环计数降至零比使其计数到某个正值`n`更为有效。
但是**如果循环计数器用作数组索引，不要这么做。
对数组的存取，数据缓存被优化为向前访问，而不是向后。**

### 复制或清除数组

将循环用于琐碎的任务，例如复制数组或将数组设置为全零，可能不是最佳选择。 例如：
```cpp
// Example 7.33a
const int size = 1000; int i;
float a[size], b[size];
// set a to zero
for (i = 0; i < size; i++) a[i] = 0.0;
// copy a to b
for (i = 0; i < size; i++) b[i] = a[i];
```

使用函数 `memset` 和 `memcpy` 一般更快一些：
```cpp
// Example 7.33b
const int size = 1000;
float a[size], b[size];
// set a to zero
memset(a, 0, sizeof(a));
// copy a to b
memcpy(b, a, sizeof(b));
```

至少在简单的情况下，大多数编译器会通过调用memset和memcpy自动替换这样的循环。
显式使用`memset`和`memcpy`是不安全的，因为如果size参数大于目标数组，则可能发生严重错误。
但是如果循环计数太大，循环也会发生相同的错误。